In [1]:
#Importing Modules

import os
import csv
import cv2
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle
import sklearn
import tensorflow as tf
tf.python.control_flow_ops = tf
%matplotlib inline

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, InputLayer, Lambda, Dropout
from keras.layers import Convolution2D, MaxPooling2D, Flatten, SpatialDropout2D, Cropping2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

%matplotlib inline

Using TensorFlow backend.


In [2]:
#Read track 1 training data

lines = []
with open('track1_data\driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.3)

print("Number of training samples = {}".format(len(train_samples)))
print("Number of validation samples = {}".format(len(validation_samples)))

Number of training samples = 4594
Number of validation samples = 1969


In [3]:
def CropImage(image):
    image = image[60:140,40:280]
    return cv2.resize(image, (200, 66))

# Cropped Image Size
cropped_image_rows, cropped_image_cols = 66, 200

In [4]:
def generator(samples, batch_size=128):
    num_samples = len(samples)
    steeringCorrection = 0.2
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = 'track1_data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = load_img(name)
                center_image_rgb = CropImage(cv2.cvtColor(img_to_array(center_image), cv2.COLOR_RGB2YUV))
                center_angle = float(batch_sample[3])
                images.append(center_image_rgb)
                angles.append(center_angle)
                

                #flip image
                center_image_flip = np.fliplr(center_image_rgb)
                center_angle_flip = -center_angle
                images.append(center_image_flip)
                angles.append(center_angle_flip)

    
                #Read left image
                name = 'track1_data/IMG/'+batch_sample[1].split('/')[-1]
                left_image = load_img(name)
                left_image_rgb = CropImage(cv2.cvtColor(img_to_array(left_image), cv2.COLOR_RGB2YUV))
                left_angle = center_angle + steeringCorrection
                images.append(left_image_rgb)
                angles.append(left_angle)
                

                #flip image
                left_image_flip = np.fliplr(left_image_rgb)
                left_angle_flip = -left_angle
                images.append(left_image_flip)
                angles.append(left_angle_flip)

                
                #Read right image
                name = 'track1_data/IMG/'+batch_sample[2].split('/')[-1]
                right_image = load_img(name)
                right_image_rgb = CropImage(cv2.cvtColor(img_to_array(right_image), cv2.COLOR_RGB2YUV))
                right_angle = center_angle - steeringCorrection
                images.append(right_image_rgb)
                angles.append(right_angle)
                
        
                #flip image
                right_image_flip = np.fliplr(right_image_rgb)
                right_angle_flip = -right_angle
                images.append(right_image_flip)
                angles.append(right_angle_flip)
        
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
            

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=128)
validation_generator = generator(validation_samples, batch_size=128)

In [5]:
#Nvidia Architecture

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(cropped_image_rows, cropped_image_cols, 3), name='NORM'))
model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu', name='CONV1'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu', name='CONV2'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu', name='CONV3'))
model.add(Convolution2D(64, 3, 3, activation='relu', name='CONV4'))
model.add(Convolution2D(64, 3, 3, activation='relu', name='CONV5'))
model.add(Flatten())
model.add(Dense(1164, activation='relu', name='FC1'))
model.add(Dense(100, activation='relu', name='FC2'))
model.add(Dense(50, activation='relu', name='FC3'))
model.add(Dense(10, activation='relu', name='FC4'))
model.add(Dense(1, name='output'))
model.summary()

opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='mse', metrics=[])
model.fit_generator(train_generator, samples_per_epoch=6*len(train_samples), validation_data=validation_generator, nb_val_samples=6*len(validation_samples), nb_epoch=5)

model.save('model.h5')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
NORM (Lambda)                    (None, 66, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
CONV1 (Convolution2D)            (None, 31, 98, 24)    1824        NORM[0][0]                       
____________________________________________________________________________________________________
CONV2 (Convolution2D)            (None, 14, 47, 36)    21636       CONV1[0][0]                      
____________________________________________________________________________________________________
CONV3 (Convolution2D)            (None, 5, 22, 48)     43248       CONV2[0][0]                      
___________________________________________________________________________________________

KeyboardInterrupt: 